In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

# ✅ CONFIGURATION
image_dir = "/kaggle/input/original/train/images"
label_dir = "/kaggle/input/original/train/labels"
bg_image_dir = "/kaggle/input/background/background_only"
bg_label_dir = "/kaggle/input/background/background_only_labels"
output_base = "/kaggle/working/combined_split"
num_bg_to_add = 180  # 👈 change this as needed

# ✅ Step 1: Get only labeled image base names (non-empty label .txts)
image_basenames = []
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        with open(os.path.join(label_dir, fname)) as f:
            if f.read().strip():
                image_basenames.append(fname.replace(".txt", ""))

# ✅ Step 2: Split labeled data
train_val, test = train_test_split(image_basenames, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.2222, random_state=42)  # ~20% val

# ✅ Step 3: Prepare output folders
def copy_split(split_names, split_type):
    img_out = os.path.join(output_base, split_type, "images")
    lbl_out = os.path.join(output_base, split_type, "labels")
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    for name in split_names:
        shutil.copy(os.path.join(image_dir, f"{name}.jpg"), os.path.join(img_out, f"{name}.jpg"))
        shutil.copy(os.path.join(label_dir, f"{name}.txt"), os.path.join(lbl_out, f"{name}.txt"))

copy_split(train, "train")
copy_split(val, "valid")
copy_split(test, "test")

# ✅ Step 4: Select & copy a limited number of background-only images to training set
bg_images = [f for f in os.listdir(bg_image_dir) if f.endswith(".jpg")]
random.shuffle(bg_images)
bg_images_to_add = bg_images[:num_bg_to_add]

train_img_dir = os.path.join(output_base, "train/images")
train_lbl_dir = os.path.join(output_base, "train/labels")

for img_name in bg_images_to_add:
    base = os.path.splitext(img_name)[0]
    img_path = os.path.join(bg_image_dir, img_name)
    lbl_path = os.path.join(bg_label_dir, f"{base}.txt")
    shutil.copy(img_path, os.path.join(train_img_dir, img_name))
    shutil.copy(lbl_path, os.path.join(train_lbl_dir, f"{base}.txt"))

print(f"✅ Final split: {len(train)} train (+{len(bg_images_to_add)} bg), {len(val)} val, {len(test)} test")

✅ Final split: 346 train (+120 bg), 99 val, 50 test


In [2]:
!pip install ultralytics --upgrade --quiet

import os
from ultralytics import YOLO


DATA_YAML_PATH = "/kaggle/input/yaml-file/data.yaml"  
model = YOLO("yolov8s.pt").to("cuda")  

# Train the model
model.train(
    data=DATA_YAML_PATH,
    device='cuda',
    epochs=100,
    batch=8,
    imgsz=896,         
    patience=20,
    optimizer='SGD',
    lr0=0.002,
    lrf=0.1,
    weight_decay=0.0005,
    cos_lr=True,
    save_period=10,
    workers=4,
    amp=True,

    mosaic=1.0,
    auto_augment='randaugment',
    augment=True,

    hsv_h=0.01,
    hsv_s=0.8,
    hsv_v=0.4,

    flipud=0.5,
    fliplr=0.5,

    translate=0.1,
    scale=0.3,
    shear=0.05,
    perspective=0.001,
    degrees=5.0,

    mixup=0.1,
    erasing=0.0,
    close_mosaic=2,
    dropout=0.1,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00:00:0100:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.c

100%|██████████| 18.4M/18.4M [00:00<00:00, 167MB/s]


Ultralytics 8.3.156 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=2, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/yaml-file/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.8, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, perspective=0.001, plots=True, pose=12.0, pretrained=T

100%|██████████| 755k/755k [00:00<00:00, 17.7MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

...

                   all         99       1181      0.994      0.933      0.948      0.649



100 epochs completed in 0.498 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.39it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         99       1181       0.99      0.926      0.968      0.661
          Absence_Head         31         31      0.994      0.742      0.896      0.537
          Absence_Tail         29         29      0.989      0.966      0.987      0.739
            Head_Rivet         99        560      0.998      0.996      0.995      0.698
            Tail_Rivet         99        561      0.979      0.999      0.994      0.671
Speed: 0.4ms preprocess, 27.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fe03d048110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [4]:
import shutil

# Copy data.yaml from input to working directory
shutil.copy("/kaggle/input/yaml-file/data.yaml", "/kaggle/working/combined_split/data.yaml")

'/kaggle/working/combined_split/data.yaml'

In [6]:
!pip install ultralytics --upgrade --quiet
import os
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Step 1: Define paths
weights_path = "/kaggle/working/runs/detect/train/weights/best.pt"  # Change this to your actual path
predict_dir = "/kaggle/working/predict"
test_images_dir = "/kaggle/working/combined_split/test/images"
yaml_path = "/kaggle/working/combined_split/data.yaml"

# Step 2: Create predict folder
os.makedirs(predict_dir, exist_ok=True)

# Step 3: Copy weights to working directory if needed
working_weights_path = "/kaggle/working/runs/detect/train/weights/best.pt"
if not os.path.exists(working_weights_path):
    shutil.copy(weights_path, working_weights_path)

# Step 4: Load the model
model = YOLO(working_weights_path)

# Step 5: Run prediction on test images
results = model.predict(
    source=test_images_dir,
    save=True,
    save_txt=True,
    conf=0.50,
    imgsz=896,
    project=predict_dir,
    name="predict_results",
    exist_ok=True
)

# Step 6: Run validation on test set to get confusion matrix
metrics = model.val(
    data=yaml_path,
    split='test',
    save_json=True,
    plots=True,
    project=predict_dir,
    name="val_results",
    exist_ok=True
)

print("✅ Inference complete.")


image 1/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_1.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 2/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_3.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 3/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 4/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e_aug_2.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 26.4ms
image 5/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b115382dd6b58dc81aad5.jpg: 896x896 6 Head_Rivets, 6 Tail_Rivets, 24.5ms
image 6/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b1153

val: Scanning /kaggle/working/combined_split/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 1325.05it/s]

val: New cache created: /kaggle/working/combined_split/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.04it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50        600      0.979          1      0.995      0.697
          Absence_Head          7          7      0.964          1      0.995      0.651
          Absence_Tail         20         20      0.963          1      0.995      0.742
            Head_Rivet         50        292          1          1      0.995      0.716
            Tail_Rivet         50        281      0.988          1      0.995      0.679
Speed: 10.7ms preprocess, 29.5ms inference, 0.0ms loss, 14.2ms postprocess per image
Saving /kaggle/working/predict/val_results/predictions.json...
Results saved to /kaggle/working/predict/val_results
✅ Inference complete.


In [7]:
import shutil

# Zip the predict_tta folder
shutil.make_archive("/kaggle/working/predict/predict_results", 'zip', "/kaggle/working/predict")
print("✅ Folder zipped as /kaggle/working/predict.zip")

✅ Folder zipped as /kaggle/working/predict.zip
